In [1]:
import PyQt5
%matplotlib qt
import pyzed.sl as sl
from ZED import ZED
from TRACKING import handtracking
import cv2
import matplotlib.pyplot as plt
import numpy as np
import sys

In [2]:
LIVE_STREAM = True
svo_file_name = "test.svo"
if LIVE_STREAM == False:
    if svo_file_name is None:
        print('Undefined .svo file name for processing')
        sys.exit()

In [3]:
# Run this Block to Run Hand Tracking with LIVE STREAM VIDEO
if LIVE_STREAM == True:
    zed = ZED(fps=60)
    zed.config_resolution("HD720")
    zed.config_depth_mode("NEURAL")
    zed.open_camera(fill_mode=True)
    zed.get_calibration_params()
    fx = zed.fx
    fy = zed.fy
    cx = zed.cx
    cy = zed.cy
    width = zed.camera.get_camera_information().camera_configuration.resolution.width
    height = zed.camera.get_camera_information().camera_configuration.resolution.height

Using Live Stream from ZED
Open ZED Successfully!


In [4]:
# Run this Block to Run Hand Tracking with RECORDED VIDEO (.SVO FILE)
# And Specify .svo File Name
if LIVE_STREAM == False:
    input_name = svo_file_name
    zed = ZED(fps=None, input_filename = input_name)
    zed.config_depth_mode("NEURAL")
    zed.open_camera(fill_mode=True)
    zed.get_calibration_params()
    fx = zed.fx
    fy = zed.fy
    cx = zed.cx
    cy = zed.cy
    width = zed.camera.get_camera_information().camera_configuration.resolution.width
    height = zed.camera.get_camera_information().camera_configuration.resolution.height
    svo_frame_rate = zed.camera.get_init_parameters().camera_fps
    num_frames = zed.camera.get_svo_number_of_frames()
    print("The SVO File's Width Is: ", width)
    print("The SVO File's Height Is: ", height)
    print("The SVO File's FPS is: ", svo_frame_rate)
    print("The SVO File's Number of Frames is: ", num_frames)


In [5]:
detector = handtracking()

In [6]:
fig = plt.figure()
plt.ion()
ax = fig.add_subplot(111, projection='3d')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [7]:
while True:
    zed.camera.grab(zed.runtime_params)
    zed.get_image()
    image = zed.image_cv2
    point_cloud = zed.point_cloud
    image = detector.detect_landmarks(image)
    data_left, data_right = detector.get_position(image, point_cloud, fx = fx, fy = fy, width=width, height=height)

    if data_right.shape == (21,3) and data_left.shape == (21,3):
        data_plot = np.vstack((data_right,data_left))
    elif data_right.shape == (21,3):
        data_plot = data_right
    elif data_left.shape == (21,3):
        data_plot = data_left
    else:
        data_plot = np.array([])
    detector.plot(ax,plt,data_plot)

    cv2.imshow("test", image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
zed.camera.close()

/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:40: RuntimeWarning: invalid value encountered in scalar multiply
  x_3d = wrist_position[0] + (landmark.x * width - wrist_landmark[0] * width) * wrist_position[2]/fx
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:42: RuntimeWarning: invalid value encountered in scalar multiply
  y_3d = wrist_position[1] + (landmark.y * height - wrist_landmark[1] * height) * wrist_position[2]/fy
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:40: RuntimeWarning: invalid value encountered in scalar add
  x_3d = wrist_position[0] + (landmark.x * width - wrist_landmark[0] * width) * wrist_position[2]/fx
/home/drowland-admin/yzheng/hand_tracking/TRACKING.py:42: RuntimeWarning: invalid value encountered in scalar add
  y_3d = wrist_position[1] + (landmark.y * height - wrist_landmark[1] * height) * wrist_position[2]/fy
